In [1]:
import pandas as pd
import glob
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [3]:
!pip3 install torch==1.10.1+cpu torchvision==0.11.2+cpu torchaudio==0.10.1+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cpu/torch_stable.html


In [4]:
!pip install transformers

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

from transformers import pipeline

In [3]:
import warnings
warnings.filterwarnings("ignore")


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
!pip install pyarrow


### ConversationsMentioningUniversities refers to tweets mentioning those Universities 

In [21]:
ConversationsMentioningUniversities= pd.concat(map(pd.read_feather, glob.glob('../DataStore/ConversationsMentioningUniversities*.ftr')),ignore_index=True).sample(50)

In [22]:
#### We transform time-related columms to pandas datetime format

In [23]:
ConversationsMentioningUniversities['created_at']=pd.to_datetime(ConversationsMentioningUniversities['created_at'])
ConversationsMentioningUniversities['collection_timestamp']=pd.to_datetime(ConversationsMentioningUniversities['created_at']).dt.strftime('%Y-%m-%d')

### Let's compute the emotions associated with each tweet

In [24]:
tokenizer = AutoTokenizer.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")

model = AutoModelForSequenceClassification.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")

In [25]:
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

### Let's clean text a bit to improve emotion extraction

In [26]:
ConversationsMentioningUniversities['text.original']=ConversationsMentioningUniversities['text'].str.replace(r'\n', '').str.replace(r'#', '').str.replace(r'@', '').str.replace(r'http\S+', '', regex=True).str.replace(r'&amp;', '&')


ConversationsMentioningUniversities['text.original']=ConversationsMentioningUniversities['text.original'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii') if x is not None else x)


### Let's compute emotions

In [27]:
import time
start_time = time.time()
ConversationsMentioningUniversities['text.classification']=ConversationsMentioningUniversities['text.original'].apply(lambda x:classifier(x))

print("--- %s seconds ---" % (time.time() - start_time))

--- 8.826875448226929 seconds ---


In [28]:
ConversationsMentioningUniversities['emotion']=ConversationsMentioningUniversities['text.classification'].apply(lambda x:x[0]['label'])

### Highly amplified and negative conversations

In [29]:
ConversationsMentioningUniversities[ConversationsMentioningUniversities['emotion'].str.contains('anger|fear|sadness')].sort_values(by='public_metrics.retweet_count',ascending=False)[['text','public_metrics.retweet_count','emotion']]

,text,public_metrics.retweet_count,emotion
3092,RT @sophiassocials: 7 years ago today I got an...,453,anger
2545,RT @sophiassocials: 7 years ago today I got an...,453,anger
4510,RT @sophiassocials: 7 years ago today I got an...,453,anger
2000,RT @UniofOxford: Puddles in Oxford 😍\n\n📷 | Sp...,108,fear
9567,RT @RalserLab: https://t.co/LnWpwgcgKM Tour de...,9,fear
6285,RT @FollowPhDs: Ok\n\nSo what is your opinion?...,2,anger
4441,RT @MLGG2: @rick_pittman @WHO @OttawaHealth @O...,2,anger
2155,RT @LIDC_UK: According to @LSHTM and @ucl rese...,2,fear
13091,RT @BlueKnight1878: @hughosmond Imagine workin...,1,sadness
10114,@Femi_Sorry @sajidjavid @UniofOxford @AstraZen...,0,anger
